In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os
import time
import cv2

In [2]:
#loading
o_data=np.load("merge\\preprocess_turi_slices_image.npy")
label_data=pd.read_csv("merge\\preprocess_turi_slices_label.csv")
o_label_data=label_data.cancer.as_matrix()

In [3]:
#create iterator for batchs
class batch_data_generater:
    def __init__(self,X,y,batchsize=10,batch_type="random",split=False):
    
                                  
        self.X=X
        self.y=self.label_transfer(y)
        self.batchsize=batchsize
        self.batch_type=batch_type
        self.index=0
        if split!=False:
            
            if type(split) is float:
                self.X,self.X_test,self.y,self.y_test=train_test_split(self.X, self.y, test_size=split, random_state=0 )
            else:
                raise ValueError("split must be float")
        if batchsize > len(self.X) or batchsize<0:
            
            raise ValueError("batchsize error")  
    def get_test(self):
        return self.X_test.astype("float32"),self.y_test.astype("float32")
        
                
    
    def label_transfer(self,y):
        new=[]
        y_map={}
        classes=[]
        for class_ in y:
            if class_ not in classes:
                classes.append(class_)
        classes.sort()
        if len(classes)<=1:
            raise ValueError("label error")
        for i in range(len(classes)):
            y_map.update({classes[i]:i})
        
        new_y=np.zeros([len(y),len(y_map)])
        for i in range(len(y)):
            new_y[i][y_map[y[i]]]=1    
        return new_y
    
    def iter_test(self):
        #global index
        File_length=len(self.X)
        end_file=self.index+self.batchsize
        batch_index=[]
        if end_file >File_length:
            end_file=File_length
            for i in range(self.index,end_file):
                batch_index.append(i)
                
            self.index=0
            end_file=end_file-File_length
            
        for i in range(self.index,end_file):
                batch_index.append(i)
        if end_file == File_length:
            end_file=0
        self.index=end_file
        yield self.X[batch_index],self.y[batch_index]
        
    def iter_test_random(self):
    
        batch_index = 0
        while True:
            # shuffle labels and features
            batch_index = np.arange(0, len(self.X))
            
            np.random.shuffle(batch_index)

            shuf_features = self.X[batch_index]
            
            shuf_labels = self.y[batch_index]
            
            for batch_idx in range(0, len(self.X), self.batchsize):
                
                images_batch = shuf_features[batch_idx:batch_idx+self.batchsize]
                images_batch = images_batch.astype("float32")
                labels_batch = shuf_labels[batch_idx:batch_idx+self.batchsize]

                yield images_batch, labels_batch
                
        
    
    def get_batch(self):
        if self.batch_type=="step":
            iter_style=self.iter_test()
            X,y=next(iter_style)
        elif self.batch_type=="random":
            iter_style=self.iter_test_random()
            X,y=next(iter_style)
        elif self.batch_type=="echo":
            X=self.X
            y=self.y
        else:
            raise ValueError("batch type error:(step,random,echo)")
        return X,y
        

In [4]:
#setting_environment

def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
 
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result


def weight_variable(shape,name):
    initial = tf.truncated_normal(shape, stddev=0.1,name=name)
    return tf.Variable(initial)

def bias_variable(shape,name):
    initial = tf.constant(0.1, shape=shape,name=name)
    return tf.Variable(initial)

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1, 1, 1, 1], padding='SAME')

def max_pool_2x2x2(x):
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')
def max_pool_2x1x1(x):
    return tf.nn.max_pool3d(x, ksize=[1,2,1,1,1], strides=[1,2,1,1,1], padding='SAME')
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32,[None, 64,32,32], name='x_input') 
    ys = tf.placeholder(tf.float32,[None, 2],name='y_input')
    learning_rate = tf.placeholder(tf.float32,[])
    x_image = tf.reshape(xs, [-1,64,32,32, 1])
    keep_prob = tf.placeholder(tf.float32,[])
with tf.name_scope('conv1_layer'):
## conv1 layer ##

    W_conv1 = weight_variable([3,3,3,1,16],"weight1") # patch 3 x 3 x 3, in 1, out 16 
    tf.histogram_summary("weight1", W_conv1)
    b_conv1 = bias_variable([16],"bias1")
    tf.histogram_summary("bias1", b_conv1)
    h_conv1 = tf.nn.relu(conv3d(x_image, W_conv1) + b_conv1) 
    h_pool1 = max_pool_2x2x2(h_conv1) # 32 x 16 x 16 x 16
with tf.name_scope('conv2_layer'):
    ## conv2 layer ##
    W_conv2 = weight_variable([3,3,3, 16, 32],"weight2") # patch 3 x 3 x 3, in 16, out 32 
    
    b_conv2 = bias_variable([32],"bias2")
    tf.histogram_summary("bias2", b_conv2)
    h_conv2 = tf.nn.relu(conv3d(h_pool1, W_conv2) + b_conv2) 
    h_pool2 = max_pool_2x2x2(h_conv2)   #16 x 8 x 8 x 32
with tf.name_scope('conv3_layer'):
    ## conv3 layer ##
    W_conv3 = weight_variable([3,3,3, 32, 64],"weight3") # patch 3 x 3 x 3, in 32, out 64 
    
    b_conv3 = bias_variable([64],"bias3")
    tf.histogram_summary("bias3", b_conv3)
    h_conv3 = tf.nn.relu(conv3d(h_pool2, W_conv3) + b_conv3) 
    h_pool3 = max_pool_2x2x2(h_conv3)   # 8 x 4 x 4 x 64
with tf.name_scope('fc_layer1'):    
## fc1 layer ##
    W_fc1 = weight_variable([8*4*4*64, 128],"fc_weight1")
    
    b_fc1 = bias_variable([128],"fcbias1")
    tf.histogram_summary("fcbias1", b_fc1)
    h_pool3_flat = tf.reshape(h_pool3, [-1,8*4*4*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
with tf.name_scope('fc_layer2'):    
## fc2 layer ##
    W_fc2 = weight_variable([128, 2],"fc_weight2")
    b_fc2 = bias_variable([2],"fcbias2")
with tf.name_scope('prediction'):      
    prediction = tf.nn.sigmoid(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
# the error between prediction and real data
with tf.name_scope('loss'):
    #loss = tf.reduce_mean(tf.square(tf.sub(prediction,ys)))  
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction+1e-50)))  
    #cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction, ys),name="cross_entropy")
    tf.scalar_summary("cross_entropy",cross_entropy)   
with tf.name_scope('train'): 
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)#start at 1e-4
sess = tf.Session()
merge=tf.merge_all_summaries()
saver = tf.train.Saver()
if int((tf.__version__).split('.')[1]) < 12:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)
if int((tf.__version__).split('.')[1]) < 12:  # tensorflow version < 0.12
    writer = tf.train.SummaryWriter('log2', sess.graph)
else: # tensorflow version >= 0.12
    writer = tf.summary.FileWriter("log2", sess.graph)    

Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on their scope.
Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on their scope.
Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on their scope.
Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on their scope.
Instructions for updating:
Please switch to tf.summary.histogram. Note that 

In [5]:
#setting btach,lr,saving path,
batchData=batch_data_generater(o_data,o_label_data,1,split=0.1)
lr=1e-4
path_save="regre"

#early_start [m,n] run m different start environments, train n steps to get the best start environment
early_start=[3,3]

In [6]:
for i in range(early_start[0]):
    total_loss=0
    for j in range(early_start[1]):
        batch_data,batch_label=batchData.get_batch()
        _,loss,pred=sess.run([train_step,cross_entropy,prediction], feed_dict={xs: batch_data, ys: batch_label,learning_rate:lr, keep_prob: 0.5})
        total_loss+=loss
        if i ==0:
            loss_sum=total_loss
            saver.save(sess, path_save)
        else:
            if total_loss<loss_sum:
                saver.save(sess, path_save)
    if int((tf.__version__).split('.')[1]) < 12:
        init = tf.initialize_all_variables()
    else:
        init = tf.global_variables_initializer()
    sess.run(init)
            

In [7]:
#load the best start environment
saver.restore(sess, path_save)
#setting train state
#Maximum n steps
training_steps=10
#check arc after every n steps
checking=3
#early_stop loss
early_stop_loss=0.001
#decrease lr after every n steps
lr_decrease_steps=2
total_loss=0

In [8]:
for i in range(training_steps):
    batch_data,batch_label=batchData.get_batch()
    _,loss=sess.run([train_step,cross_entropy], feed_dict={xs: batch_data, ys: batch_label,learning_rate:lr, keep_prob: 0.5})
    total_loss+=loss
    if i%checking==0 and i!=0:
       
        x_test,y_test=batchData.get_test()
        result = sess.run(merge,feed_dict={xs: x_test, ys: y_test,learning_rate:lr,keep_prob:1})
        writer.add_summary(result, i)
        loss_mean=total_loss/checking
        #print("accuracy:",compute_accuracy(x_test, y_test))
        if early_stop_loss>loss_mean:
            break        
    if i%lr_decrease_steps==0 and i!=0:        
        lr=lr/2
#save model
saver.save(sess, path_save)      
#out class solution
#sense_layer=sess.run([h_fc1], feed_dict={xs: batch_data, ys: batch_label,learning_rate:lr, keep_prob: 1})

'D:\\regre'

In [9]:
testdata=batch_data_generater(o_data,o_label_data,1,batch_type='echo')
test_x,test_y=testdata.get_batch()
#predict
predict=np.asarray(sess.run([prediction], feed_dict={xs:test_x, ys: test_y,learning_rate:lr, keep_prob: 1})).reshape([-1,2]).argmax(1)
